In [37]:
import numpy as np
import pandas as pd
import pickle
import os
from dotenv import load_dotenv
import requests
load_dotenv("./.env")

True

In [30]:
#I want to make three lexicons 
#a crisis lexicon of words related to mental health that are associated with crisis
#a temporal lexicon that determine urgency of the crisis
#a severity lexicon that determines the severity of the crisis

crisis_lexicon = {
    "suicidal": 10, "suicide": 10, "unalive": 10, "kill myself": 10, 
    "end it all": 10, "no way out": 9, "die": 9, "goodbye forever": 9, 
    "hopeless": 8, "self-harm": 8, "cutting": 8, "jump off": 8, 
    "pills": 7, "overdose": 7, "hang myself": 10, "drown myself": 9, 
    "bridge": 8, "gun": 9, "razor": 7, "pain is unbearable": 8, 
    "nothing matters": 7, "no point in living": 9, "life is pointless": 8, 
    "I give up": 8, "can't take this anymore": 9, "don't want to be here": 9, 
    "I'm done": 8, "no future": 7, "nobody cares": 6, "I'm exhausted": 6,
    "can't stop crying": 6, "broken beyond repair": 7, "hate myself": 7,
    "failure": 6, "worthless": 7, "empty": 6, "lost all hope": 9
}


temporal_lexicon = {
    "now": 10, "immediately": 10, "tonight": 9, "this moment": 9, 
    "right now": 10, "soon": 8, "today": 8, "can't wait": 9, 
    "urgent": 9, "this second": 10, "tomorrow": 7, "before it's too late": 9, 
    "running out of time": 9, "this week": 6, "eventually": 5, 
    "someday": 4, "whenever": 3, "later": 2
}


severity_lexicon = {
    "extremely": 10, "desperately": 10, "completely": 9, "totally": 9, 
    "absolutely": 9, "cannot handle": 9, "falling apart": 8, "breaking down": 8, 
    "barely holding on": 9, "too much to bear": 9, "can't take anymore": 10, 
    "beyond saving": 9, "drowning": 8, "collapsing": 8, "out of control": 8, 
    "spiraling": 7, "barely surviving": 7, "giving up": 8, "crushed": 7, 
    "overwhelmed": 7, "anxious": 6, "stressed": 5, "worried": 4, "sad": 3, 
    "upset": 2, "unhappy": 2
}


In [31]:
def reciprocal(x):
    return 1/x

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
import spacy
model = spacy.load("en_core_web_sm")

In [68]:
def get_response(data):
    api_key = os.getenv("JINA_API_KEY")
    URL = "https://api.jina.ai/v1/embeddings"

    resp = requests.post(URL, headers={"Authorization": f"Bearer {api_key}"}, json={"input": data, "model" : "jina-embeddings-v3", "task" : "classification"})
    print(type(resp.json()["data"][0]["embedding"]))
    return resp.json()["data"][0]["embedding"]

def calculate_mean_pool(sentences):
    embeddings = []
    for sentence in sentences:
        embeddings.append(get_response(sentence)["data"][0]["embedding"])
    return np.mean(embeddings, axis=0)

def sigmoid_function(x, k = 4.5):
    return 1+ (2 / (1 + np.exp(- k * x)))

def cosine_similarity_with_sim_diff(text_embedding, sim_diff_vec):
    return np.dot(text_embedding, sim_diff_vec) / (np.linalg.norm(text_embedding) * np.linalg.norm(sim_diff_vec))

def check_crisis_lexicon(text, lexicon):
    words = []
    for word in text.lower().split():
        if word in lexicon:
            words.append(word)
    return words

def check_dependency_lexicons(text, lexicon, model : spacy.language.Language, crisis_words):
    words = []
    for word in model(text):
        if word.lemma_ in lexicon:
            if word.head.text in crisis_words:
                words.append(word)
    return words

def inference_basic_score (inference_text):
    text_embedding = get_response(inference_text)
    return sigmoid_function(cosine_similarity_with_sim_diff(text_embedding, pickle.load(open("sim_diff.pkl", "rb"))))

def return_lexicon_score(word, lexicon):
    if word in lexicon:
        return lexicon[word]/10
    else:
        return 1

In [69]:
#Final score calc

def final_score(inference_text):
    # Get crisis words from the text
    crisis_words = check_crisis_lexicon(inference_text, crisis_lexicon)
    
    # Calculate crisis lexicon score
    crisis_score = sum(return_lexicon_score(word, crisis_lexicon) 
                      for word in crisis_words)
    
    # Get temporal dependencies and calculate temporal score
    temporal_dependencies = check_dependency_lexicons(inference_text, temporal_lexicon, 
                                                    model, crisis_words)
    temporal_score = sum(return_lexicon_score(word, temporal_lexicon) 
                        for word in temporal_dependencies)
    
    # Get severity dependencies and calculate severity score
    severity_dependencies = check_dependency_lexicons(inference_text, severity_lexicon, 
                                                    model, crisis_words)
    severity_score = sum(return_lexicon_score(word, severity_lexicon) 
                        for word in severity_dependencies) if severity_dependencies else 1
    
    # Calculate inference score
    inference_score = inference_basic_score(inference_text)
    
    # Return final score: (inference_score * severity_score) + temporal_score + crisis_score
    return (inference_score * severity_score) + temporal_score + crisis_score 

In [70]:
def main():
    text = "I feel like I am losing control and I can't take this anymore. I am completely overwhelmed and I don't know what to do. I feel like I'm drowning in my own thoughts and emotions. It's all too much to bear."
    print("Crisis Score: ", final_score(text))

if __name__ == "__main__":
    main()

<class 'list'>
Crisis Score:  2.4126721968045803
